In [1]:
import warnings
warnings.filterwarnings("ignore")
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [2]:
import json
f = open('final.json' , encoding="utf-8")
TRAIN_DATA = json.load(f)

In [3]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

 21%|██▏       | 733/3434 [00:00<00:00, 3079.67it/s]

Skipping entity


 82%|████████▏ | 2805/3434 [00:00<00:00, 4005.56it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 3434/3434 [00:00<00:00, 3439.43it/s]

Skipping entity
Skipping entity
Skipping entity


In [4]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     54.30    0.09    0.15    0.06    0.00
  0     200        688.06   5350.53   77.62   76.99   78.25    0.78
  1     400        487.02   2482.60   89.53   90.45   88.64    0.90
  2     600        375.09   1998.45   93.92   94.17   93.66    0.94
  3     800        516.33   1693.52   95.87   95.98   95.76    0.96
  5    1000        552.07   1401.03   96.72   97.11   96.32    0.97
  7    1200        662.53   1483.08   97.37   97.73   97.00    0.97
  9    1400        702.38   1385.52   97.40   97.40   97.40    0.97
 12    1600       2142.83 

[2023-01-20 13:55:20,215] [INFO] Set up nlp object from config
[2023-01-20 13:55:20,224] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-01-20 13:55:20,227] [INFO] Created vocabulary
[2023-01-20 13:55:20,228] [INFO] Finished initializing nlp object
[2023-01-20 13:55:21,273] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [6]:
nlp_ner = spacy.load("./model-best")

In [9]:
t1 = "Shipper JIASHAN YAGE HOUSEHOLD ARTICLES CO . , LTD NO.1399 , XITANG ROAD , XITANG TOWN , JIASHAN COUNTY , JIAXING CITY , ZHEJIANG PROVINCE , CHINA TEL : 0573-84569628 FAX : 0573-84569658"

t2 = "NOTIFY PARTY LIFESTYLE SOLUTIONS 6955 Mowry Ave Newark , CA 94560 UNITED STATES OF AMERICA ATTN : JOHANN ABEYESINHE TEL : 510-824-6537 "

t3 = "Notify Party CODDLE INC . 6955 MOWRY AVENUE , NEWARK , CA 94560 , USA . TEL : + 1-510-824-6500 Attn : MR . JOE OROSCO E - Mail : joe@lifestylesolutions.com "

t4 = 'CARGO LOCATED AT : ST . GEORGE WAREHOUSE OF SOUTH CAROLINA FIRMS CODE : L453 1980A TECHNOLOGY DRIVE CHARLESTON , SC 29492 '


doc = nlp_ner(t4)
for ent in doc.ents:
  print(ent.text," ->>> " , ent.label_)

CARGO LOCATED AT  ->>>  KEY
ST . GEORGE WAREHOUSE OF SOUTH CAROLINA  ->>>  VALUE
FIRMS CODE  ->>>  KEY
L453 1980A  ->>>  VALUE


In [10]:
file1 = open('test.txt', 'r')
Lines = file1.readlines()

c = 0
for line in Lines:
    doc = nlp_ner(line)
    print(line)
    for ent in doc.ents:
        print(ent.text," ->>> " , ent.label_)
    print("\n", "##########################", "\n")
    c = c+1

print(c)

NOTIFY PARTY ( It is agreed that no responsibility shall be attached to the Carrier or its Agents for failure to notify ) OPENROAD TRANSPORTATION KIMBERLY DEVONDE 288 E ELLENDALE AVE DALLAS OR 97338 UNITED STATES OF AMERICA NP > 

NOTIFY PARTY  ->>>  KEY
OPENROAD TRANSPORTATION KIMBERLY DEVONDE 288 E ELLENDALE AVE DALLAS OR 97338 UNITED STATES OF AMERICA NP  ->>>  VALUE

 ########################## 

PRE - CARRIAGE BY HELENA SCHEPERS 009N 

PRE - CARRIAGE BY  ->>>  KEY
HELENA SCHEPERS 009N 
  ->>>  VALUE

 ########################## 

PLACE OF RECEIPT LEIXOES 

PLACE OF RECEIPT  ->>>  KEY
LEIXOES 
  ->>>  VALUE

 ########################## 

OCEAN VESSEL VOYAGE NO . FLAG YM EVOLUTION 055W 

OCEAN VESSEL VOYAGE NO  ->>>  KEY
FLAG YM EVOLUTION 055W 
  ->>>  VALUE

 ########################## 

PORT OF LOADING LEIXOES 

PORT OF LOADING  ->>>  KEY
LEIXOES 
  ->>>  VALUE

 ########################## 

PORT OF DISCHARGE NORFOLK , VA 

PORT OF DISCHARGE  ->>>  KEY
NORFOLK , VA  ->>>  VALUE

 

SHIPPER 4 / F , BUILDING 4 , 277 XIGANG WEST ROAD , DIPU STREES , ANJI COUNTY , HUZH OU CITY , ZHE JIANG PROV , CHINA 

SHIPPER  ->>>  KEY
4 / F , BUILDING 4 , 277 XIGANG WEST ROAD , DIPU STREES , ANJI COUNTY , HUZH OU CITY , ZHE JIANG PROV , CHINA  ->>>  VALUE

 ########################## 

CONSIGNEE NOVILAND INTERNATIONAL LLC 4150 SHACKLEFORD RD , STE540 NORCROSS , GA 30093 UNITED STATES 

CONSIGNEE  ->>>  KEY
NOVILAND INTERNATIONAL LLC 4150 SHACKLEFORD RD , STE540 NORCROSS , GA 30093 UNITED STATES  ->>>  VALUE

 ########################## 

NOTIFY PARTY JD LOGISTICS UNITED STATES COMPANY 13200 LOOP ROAD FONTANA , CA 92337 UNITED STATES ATTN : JERRY.CHENG@JD.COM 

NOTIFY PARTY  ->>>  KEY
JD LOGISTICS UNITED STATES COMPANY 13200 LOOP ROAD FONTANA , CA 92337 UNITED STATES  ->>>  VALUE
ATTN  ->>>  KEY
JERRY.CHENG@JD.COM  ->>>  VALUE

 ########################## 

MASTER B / L NO . EGLV142101403587 

MASTER B / L NO  ->>>  KEY
EGLV142101403587 
  ->>>  VALUE

 ########################## 